# Tf-Idf
Tf-Idf 알고리즘을 이용해 책 줄거리에 기반해 책 간 유사도를 구하고, 이를 통해 비슷한 책을 추천합니다.

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import display, display_pretty, clear_output, JSON

spark = (
    SparkSession
    .builder
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .getOrCreate()
)

# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size

# 공통 데이터 위치
home_jovyan = "/home/jovyan"
work_data = f"{home_jovyan}/work/data"
work_dir=!pwd
work_dir = work_dir[0]

# 로컬 환경 최적화
spark.conf.set("spark.sql.shuffle.partitions", 5) # the number of partitions to use when shuffling data for joins or aggregations.
spark.conf.set("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")
spark

22/03/28 06:45:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
!/opt/conda/bin/python --version

print("spark.version: {}".format((spark.version)))

Python 3.9.6
spark.version: 3.1.2


In [29]:
!pip install nltk

In [36]:
!pip install konlpy

In [3]:
!pip install tweepy==3.10.0

In [4]:
book = spark.read.option("header", "true").option("inferSchema", "true").csv(f"./bookdb.csv")
book.printSchema()
book.show(3)

root
 |-- 분야: string (nullable = true)
 |-- 분야 내 카테고리: string (nullable = true)
 |-- 서브 카테고리: string (nullable = true)
 |-- 순위: string (nullable = true)
 |-- 책이름: string (nullable = true)
 |-- 저자: string (nullable = true)
 |-- 줄거리: string (nullable = true)
 |-- 이미지: string (nullable = true)

+---------------------------------+----------------+-------------+----+------+-------------+----------------------------------+------+
|                             분야|분야 내 카테고리|서브 카테고리|순위|책이름|         저자|                            줄거리|이미지|
+---------------------------------+----------------+-------------+----+------+-------------+----------------------------------+------+
|                             소설|     나라별 소설|     영미소설|   1|대성당|레이먼드 카버|소설가 김연수의 번역으로 만나는...|  null|
| “의심의 여지 없이 레이먼드 카...|                |         null|null|  null|         null|                              null|  null|
|가장 위대한 문학적 동반자였다....|            null|         null|null|  null|         null|                           

In [5]:
book = book.select("책이름", "저자", "줄거리", "이미지")
book.show(3)

+------+-------------+----------------------------------+------+
|책이름|         저자|                            줄거리|이미지|
+------+-------------+----------------------------------+------+
|대성당|레이먼드 카버|소설가 김연수의 번역으로 만나는...|  null|
|  null|         null|                              null|  null|
|  null|         null|                              null|  null|
+------+-------------+----------------------------------+------+
only showing top 3 rows



In [7]:
book.createOrReplaceTempView("book")
spark.sql("select * from book where '책이름' is not null")
book = book.na.drop()
spark.sql("select * from book limit 5")

책이름,저자,줄거리,이미지
빌러비드,토니 모리슨,"차마 기억할 수 없고 잊을 수도 없는 과거를 ‘재기억’하다!흑인 여성으로는 최초로 노벨문학상을 수상한 살아 있는 미국문학의 대모, 토니 모리슨의 대표작 『빌러비드』. 미국 역...",https://bookthumb-phinf.pstatic.net/cover/076/035/07603523.jpg?type=m5
"주황은 고통, 파랑은 광기","질 D. 블록, 리 차일드, 니컬러스 크리스토퍼, 마이클 코널리, 제프리 디버, 조 R. 랜스데일, 게.. 외 10명","17편의 어둡고 기묘하고 매혹적인 이야기들!조이스 캐럴 오츠, 리 차일드, 마이클 코널리, 제프리 디버, 데이비드 모렐을 포함해 재능 넘치는 이야기꾼 열일곱 명이 고대 동굴벽...",https://bookthumb-phinf.pstatic.net/cover/155/109/15510910.jpg?type=m5
양들의 침묵,토머스 해리스,"“클라리스, 양들은 울음을 그쳤나?그 울음은 아마 영원히 멈추지 않을 거야.”우리 시대 최고의 작가 토머스 해리스가 돌아왔다!전 세계 수천만 독자들을 사로잡은《양들의 침묵》 ...",https://bookthumb-phinf.pstatic.net/cover/154/066/15406625.jpg?type=m5
우아한 연인,에이모 토울스,고전 문학적 배경을 누비는 현대적이고 세련된 인물들이 선사하는 독특한 감동!《모스크바의 신사》의 저자 에이모 토울스의 데뷔작 『우아한 연인』. 작가로서는 비교적 늦은 나이인 ...,https://bookthumb-phinf.pstatic.net/cover/154/695/15469542.jpg?type=m5
걸리버 여행기,조너선 스위프트,『동물농장』 조지 오웰이 극찬한 최고의 풍자문학 완역본환상적인 모험에 숨겨진 인간과 사회에 대한 신랄한 풍자풍자문학의 대가 조너선 스위프트의 『걸리버 여행기』는 걸리버의 환상...,https://bookthumb-phinf.pstatic.net/cover/153/732/15373293.jpg?type=m5


In [16]:
bookdf = book.toPandas()

In [33]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from konlpy.tag import Okt

형태소 분석 - 조사, 어미, 어간 등 제거

In [ ]:
output = []
for i, document in enumerate(bookdf['줄거리']):
    okt = Okt()
    clean_words = []
    for word in okt.pos(document, stem=True): #어간추출
        if word[1] in ['Noun', 'Verb', 'Adjective']:
            clean_words.append(word[0])
    document = ' '.join(clean_words)
    output.append(document)

불용어 제거

In [23]:
df = pd.read_csv('https://raw.githubusercontent.com/cranberryai/todak_todak_python/master/machine_learning_text/clean_korean_documents/korean_stopwords.txt', header=None)
df[0] = df[0].apply(lambda x: x.strip())
stopwords = df[0].to_numpy()

import nltk
nltk.download('punkt')
for i, document in enumerate(output):
    clean_words = [] 
    for word in nltk.tokenize.word_tokenize(document): 
        if word not in stopwords: #불용어 제거
            clean_words.append(word)    
    output[i] = ' '.join(clean_words)        
print(output[:5])

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['차마 기억 하다 없다 잊다 수도 없다 과거 재기 하다 흑인 여성 최초 노벨문학상 수상한 있다 미국문학 대모 토니 리슨 대표 작 빌다 비드 미국 역사 이해 하다 필수 흑인 노예제 현대 인종차별 이르다 넓다 스펙트럼 다루다 온 저자 작품 여성 노예 초점 맞추다 노예제 특수하다 상황 폭력 겪다 이야기 들려주다 여성 어머니 이기 성적 억압 모성애 박탈 겪다 하다 하다 흑인 여성 노예 운명 대물리다 끊기다 위해 딸 죽이다 실화 바탕 이야기 구성 하다 흑인 참혹하다 역사 재 조명 박탈 모성애 되찾다 도망 노예 과격하다 뒤틀리다 사랑 그로 인하다 자기 파괴 보여주다', '편의 어둡다 기묘하다 매혹 이야기 조이스 캐럴 오츠 리 차 일드 마이클 코 널리 제프리 디버 데이비드 모렐 포함 하다 재능 넘치다 이야기 일곱 고대 동굴 벽화 미켈란젤로 고갱 고흐 르누아르 마그리트 달리 이르다 채 작품 영감 받다 써다 내려가다 소설 엮 주황 고통 파랑 광기 예술 작품 선정 제한 두다 않다 덕 각각 작가 자유롭다 규칙 날개 삼다 기발하다 거치다 없다 상상력 펼치다 저 독창 매력 이야기 완성 하다 서로 시대 서로 재료 색채 스타일 빚 어진 미술 작품 소설 예술 통해 새롭다 목소리 생명 얻다 되어다', '클라리스 양 울음 그치다 울음 아마 영원하다 멈추다 않다 시대 최고 작가 토머스 해리스 돌아오다 전 세계 독자 사로자다 양 침묵 추다 간 주년 기념 스페셜 에디 션 전 세계 독자 밤잠 빼앗다 최고 스릴러 양 침묵 추다 간 주년 기념 스페셜 에디 션 추다 간 세기 스릴러 문학 모든 기록 치우다 작가 토머스 해리스 귀환 양 침묵 알다 아마 전 세계 절반 책 나머지 절반 영화로 제목 접 하다 책 주인공 한니발 렉터 늘다 희다 대의 식인 살인마 캐릭터 독자 뇌리 지워지다 않다 강렬하다 인상 남다 토머스 해리스 세계 최고 작가 열 올리다 소설 전 세계 폭넓다 애독자 층 확보', '고전 문학 배경 누비 현대 세련되다 인물 선사 하다 독특하다 감동 모스크바 신사 의 저자 에이 모 토울스 데뷔 작 우아하

In [25]:
append_col = {'processed_line' : output}
bookdf = bookdf.append(pd.DataFrame(append_col))

In [48]:
bookdf = bookdf[:163]
bookdf = bookdf.drop(['processed_line'], axis=1)
bookdf

,책이름,저자,줄거리,이미지
0,빌러비드,토니 모리슨,차마 기억할 수 없고 잊을 수도 없는 과거를 ‘재기억’하다!흑인 여성으로는 최초로 ...,https://bookthumb-phinf.pstatic.net/cover/076/...
1,"주황은 고통, 파랑은 광기","질 D. 블록, 리 차일드, 니컬러스 크리스토퍼, 마이클 코널리, 제프리 디버, 조...","17편의 어둡고 기묘하고 매혹적인 이야기들!조이스 캐럴 오츠, 리 차일드, 마이클 ...",https://bookthumb-phinf.pstatic.net/cover/155/...
2,양들의 침묵,토머스 해리스,"“클라리스, 양들은 울음을 그쳤나?그 울음은 아마 영원히 멈추지 않을 거야.”우리 ...",https://bookthumb-phinf.pstatic.net/cover/154/...
3,우아한 연인,에이모 토울스,고전 문학적 배경을 누비는 현대적이고 세련된 인물들이 선사하는 독특한 감동!《모스크...,https://bookthumb-phinf.pstatic.net/cover/154/...
4,걸리버 여행기,조너선 스위프트,『동물농장』 조지 오웰이 극찬한 최고의 풍자문학 완역본환상적인 모험에 숨겨진 인간과...,https://bookthumb-phinf.pstatic.net/cover/153/...
...,...,...,...,...
158,해변의 카프카 하,무라카미 하루키,23년 하루키 문학 인생의 결정체가 담긴 장편소설!무라카미 하루키 장편소설『해변의 ...,https://bookthumb-phinf.pstatic.net/cover/045/...
159,브루투스의 심장,히가시노 게이고,"<붉은 손가락>, <용의자 X의 헌신>, <비밀>의 작가, 히가시노 게이고의 초기 ...",https://bookthumb-phinf.pstatic.net/cover/030/...
160,공중그네,오쿠다 히데오,"뾰족한 물건만 보면 오금을 못 펴는 야쿠자 중간보스, 어느 날부턴가 공중그네에서 번...",https://bookthumb-phinf.pstatic.net/cover/014/...
161,인간 실격,다자이 오사무,인간의 나약함을 탁월하게 묘사하는 다자이 오사무의 작품을 새롭게 읽는다. 순수하고 ...,https://bookthumb-phinf.pstatic.net/cover/001/...


In [51]:
bookdf = pd.concat([bookdf, pd.DataFrame(append_col)], axis=1)

In [52]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(bookdf['processed_line'])
print('TF-IDF 행렬의 크기(shape) :',tfidf_matrix.shape)

TF-IDF 행렬의 크기(shape) : (163, 3633)


In [53]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('코사인 유사도 연산 결과 :',cosine_sim.shape)

코사인 유사도 연산 결과 : (163, 163)


In [54]:
title_to_index = dict(zip(bookdf['책이름'], bookdf.index))

# 책 제목 '인간 실격'의 인덱스를 리턴
idx = title_to_index['인간 실격']
print(idx)

161


In [55]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 책제목으로부터 해당 책의 인덱스를 받아온다.
    idx = title_to_index[title]

    # 해당 책 모든 책들간의 유사도
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 정렬한다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 책 받아오기
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 책 인덱스
    book_indices = [idx[0] for idx in sim_scores]

    # 가장 유사한 10개의 책 제목 리턴
    return bookdf['책이름'].iloc[book_indices]

get_recommendations('양들의 침묵')

39         해리 포터와 비밀의 방 1
37        해리 포터와 마법사의 돌 1
38     해리 포터와 마법사의 돌 2/완결
43           해리포터와 불의 잔 1
41       해리포터와 아즈카반의 죄수 1
42       해리포터와 아즈카반의 죄수 2
152                1Q84 1
8                     모비딕
44           해리포터와 불의 잔 2
45           해리포터와 불의 잔 4
Name: 책이름, dtype: object

mysql로 테이블 저장

In [ ]:
bookdf.write.mode("overwrite").jdbc("jdbc:mysql://mysql:3306/bookdb", "bookdb.books", properties={"user":"root", "password":"root"})